### Limpieza de los datos

In [2]:
import pandas as pd
import numpy as np
import os
from joblib import dump

Antes que nada, debemos tener en cuenta que el conjunto de datos incluye múltiples datasets (uno para cada año desde 1991 a 2023), por lo que primero debemos unirlos en uno solo.

In [3]:
years = range(1991, 2024) 

dfs = {}

for year in years:
    file_path = f'atp_matches_{year}.csv'
    dfs[f'df_{year}'] = pd.read_csv(file_path)

df = pd.concat(dfs.values(), ignore_index=True)

In [4]:
df.to_csv('raw_data.csv')

In [5]:
df.head(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1991-339,Adelaide,Hard,32,A,19901231,1,101723,NaN,NaN,...,62.0,44.0,23.0,16.0,6.0,8.0,56.0,NaN,2.0,NaN
1,1991-339,Adelaide,Hard,32,A,19901231,2,100946,NaN,Q,...,41.0,35.0,27.0,15.0,1.0,2.0,304.0,NaN,75.0,NaN
2,1991-339,Adelaide,Hard,32,A,19901231,3,101234,NaN,NaN,...,37.0,22.0,6.0,8.0,4.0,8.0,82.0,NaN,69.0,NaN
3,1991-339,Adelaide,Hard,32,A,19901231,4,101889,8.0,NaN,...,45.0,30.0,11.0,10.0,5.0,8.0,50.0,NaN,84.0,NaN
4,1991-339,Adelaide,Hard,32,A,19901231,5,101274,NaN,NaN,...,41.0,28.0,15.0,11.0,4.0,8.0,88.0,NaN,28.0,NaN


In [15]:
df['winner_ht'].min()

160.0

In [49]:
df.shape

(105269, 49)

El dataset inicial tiene 105.269 filas y 49 columnas.

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105269 entries, 0 to 105268
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          105269 non-null  object 
 1   tourney_name        105269 non-null  object 
 2   surface             105269 non-null  object 
 3   draw_size           105269 non-null  int64  
 4   tourney_level       105269 non-null  object 
 5   tourney_date        105269 non-null  int64  
 6   match_num           105269 non-null  int64  
 7   winner_id           105269 non-null  int64  
 8   winner_seed         42625 non-null   float64
 9   winner_entry        12894 non-null   object 
 10  winner_name         105269 non-null  object 
 11  winner_hand         105260 non-null  object 
 12  winner_ht           102747 non-null  float64
 13  winner_ioc          105269 non-null  object 
 14  winner_age          105262 non-null  float64
 15  loser_id            105269 non-nul

In [51]:
df.isnull().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           62644
winner_entry          92375
winner_name               0
winner_hand               9
winner_ht              2522
winner_ioc                0
winner_age                7
loser_id                  0
loser_seed            81833
loser_entry           84066
loser_name                0
loser_hand               42
loser_ht               4948
loser_ioc                 0
loser_age                24
score                     2
best_of                   0
round                     0
minutes               13148
w_ace                 10320
w_df                  10320
w_svpt                10320
w_1stIn               10320
w_1stWon              10320
w_2ndWon              10320
w_SvGms               10319
w_bpSaved             10320
w_bpFaced           

Primero vamos a ocuparnos de los NaNs. Teniendo en cuenta que hay un total de 105.269 registros, vemos que las columnas winner_seed, winner_entry, loser_seed y loser_entry tienen valores nulos en su mayoría, de modo que las eliminamos. 

In [52]:
df.drop(['winner_seed', 'winner_entry', 'loser_seed', 'loser_entry'], axis=1, inplace=True)
df.isnull().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_name               0
winner_hand               9
winner_ht              2522
winner_ioc                0
winner_age                7
loser_id                  0
loser_name                0
loser_hand               42
loser_ht               4948
loser_ioc                 0
loser_age                24
score                     2
best_of                   0
round                     0
minutes               13148
w_ace                 10320
w_df                  10320
w_svpt                10320
w_1stIn               10320
w_1stWon              10320
w_2ndWon              10320
w_SvGms               10319
w_bpSaved             10320
w_bpFaced             10320
l_ace                 10320
l_df                  10320
l_svpt                10320
l_1stIn             

Seguimos trabajando con los NaNs, empezando por los atributos con más valores nulos. En este caso la variable 'minutes'. 

In [53]:
df['minutes'].describe()

count    92121.000000
mean       103.870757
std         39.513006
min          0.000000
25%         75.000000
50%         97.000000
75%        125.000000
max       1146.000000
Name: minutes, dtype: float64

Vemos que la distribución de los valores tiende a la distribución normal, con una media muy próxima a la mediana y poca desviación, así que imputamos la duración media de los partidos en los valores nulos.

In [54]:
df['minutes'] = df['minutes'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_name               0
winner_hand               9
winner_ht              2522
winner_ioc                0
winner_age                7
loser_id                  0
loser_name                0
loser_hand               42
loser_ht               4948
loser_ioc                 0
loser_age                24
score                     2
best_of                   0
round                     0
minutes                   0
w_ace                 10320
w_df                  10320
w_svpt                10320
w_1stIn               10320
w_1stWon              10320
w_2ndWon              10320
w_SvGms               10319
w_bpSaved             10320
w_bpFaced             10320
l_ace                 10320
l_df                  10320
l_svpt                10320
l_1stIn             

Seguimos con las variables relativas a los puntos de los jugadores en el partido, de la columna 'w_ace' (servicios directos del ganador) a la columna 'l_bpFaced' (puntos de rotura a los que se ha enfrentado el perdedor). Vemos que cuando los atributos tienen NaNs los tienen en su conjunto, es decir, o bien se han recogido todos los datos durante el partido o no se ha recogido ninguno. 

Teniendo en cuenta que se trata de variables muy significativas para el modelo, y que los registros con NaNs no alcanzan el 10% del conjunto de datos, optamos por eliminar las filas en las que estas columnas tienen valores nulos.

In [55]:
columns_to_drop_na = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 
                      'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 
                      'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced']

df.dropna(subset=columns_to_drop_na, inplace=True)
df.isnull().sum()

tourney_id               0
tourney_name             0
surface                  0
draw_size                0
tourney_level            0
tourney_date             0
match_num                0
winner_id                0
winner_name              0
winner_hand              5
winner_ht              373
winner_ioc               0
winner_age               0
loser_id                 0
loser_name               0
loser_hand              24
loser_ht              1269
loser_ioc                0
loser_age               17
score                    0
best_of                  0
round                    0
minutes                  0
w_ace                    0
w_df                     0
w_svpt                   0
w_1stIn                  0
w_1stWon                 0
w_2ndWon                 0
w_SvGms                  0
w_bpSaved                0
w_bpFaced                0
l_ace                    0
l_df                     0
l_svpt                   0
l_1stIn                  0
l_1stWon                 0
l

Seguimos con los atributos relativos a los puntos de clasificación de los jugadores. Teniendo en cuenta que el dato relevante para el modelo es el ranking del jugador, y no los puntos, eliminamos las columnas 'winner_rank_points' y 'loser_rank_points'.

In [56]:
df.drop(['winner_rank_points', 'loser_rank_points'], axis=1, inplace=True)
df.isnull().sum()

tourney_id          0
tourney_name        0
surface             0
draw_size           0
tourney_level       0
tourney_date        0
match_num           0
winner_id           0
winner_name         0
winner_hand         5
winner_ht         373
winner_ioc          0
winner_age          0
loser_id            0
loser_name          0
loser_hand         24
loser_ht         1269
loser_ioc           0
loser_age          17
score               0
best_of             0
round               0
minutes             0
w_ace               0
w_df                0
w_svpt              0
w_1stIn             0
w_1stWon            0
w_2ndWon            0
w_SvGms             0
w_bpSaved           0
w_bpFaced           0
l_ace               0
l_df                0
l_svpt              0
l_1stIn             0
l_1stWon            0
l_2ndWon            0
l_SvGms             0
l_bpSaved           0
l_bpFaced           0
winner_rank       280
loser_rank        615
dtype: int64

Nos centramos ahora en los atributos 'winner_rank' y 'loser_rank'. Un jugador puede cambiar de posición en la clasificación constantemente, pero los cambios en el tiempo no suelen ser muy bruscos. Es decir, si en enero de 2003 el jugador está en la 4ª posición raramente en febrero de 2003 estará en la 40ª, a no ser que haya sufrido una lesión o por alguna otra causa justificada. Por tanto, para tratar estos NaNs imputaremos la posición del mismo jugador en la fecha más próxima.

Para ello, ordenamos primero los registros por nombre del jugador y fecha, y seguidamente imputamos.

In [57]:
df_sorted = df.sort_values(by=['winner_name', 'tourney_date'])

df['winner_rank'] = df_sorted.groupby('winner_name')['winner_rank'].fillna(method='ffill')
df['loser_rank'] = df_sorted.groupby('loser_name')['loser_rank'].fillna(method='ffill')

print(df.isna().sum())
df = df.sort_index()

tourney_id          0
tourney_name        0
surface             0
draw_size           0
tourney_level       0
tourney_date        0
match_num           0
winner_id           0
winner_name         0
winner_hand         5
winner_ht         373
winner_ioc          0
winner_age          0
loser_id            0
loser_name          0
loser_hand         24
loser_ht         1269
loser_ioc           0
loser_age          17
score               0
best_of             0
round               0
minutes             0
w_ace               0
w_df                0
w_svpt              0
w_1stIn             0
w_1stWon            0
w_2ndWon            0
w_SvGms             0
w_bpSaved           0
w_bpFaced           0
l_ace               0
l_df                0
l_svpt              0
l_1stIn             0
l_1stWon            0
l_2ndWon            0
l_SvGms             0
l_bpSaved           0
l_bpFaced           0
winner_rank       209
loser_rank        408
dtype: int64


Vemos que siguen habiendo NaNs en ambas variables. Dado que se trata de datos importantes para que el modelo haga las predicciones y representan una fracción muy pequeña de los datos, los eliminamos.

In [58]:
columns_to_drop_na2 = ['winner_rank', 'loser_rank']

df.dropna(subset=columns_to_drop_na2, inplace=True)
df.isnull().sum()

tourney_id          0
tourney_name        0
surface             0
draw_size           0
tourney_level       0
tourney_date        0
match_num           0
winner_id           0
winner_name         0
winner_hand         1
winner_ht         321
winner_ioc          0
winner_age          0
loser_id            0
loser_name          0
loser_hand          5
loser_ht         1098
loser_ioc           0
loser_age           9
score               0
best_of             0
round               0
minutes             0
w_ace               0
w_df                0
w_svpt              0
w_1stIn             0
w_1stWon            0
w_2ndWon            0
w_SvGms             0
w_bpSaved           0
w_bpFaced           0
l_ace               0
l_df                0
l_svpt              0
l_1stIn             0
l_1stWon            0
l_2ndWon            0
l_SvGms             0
l_bpSaved           0
l_bpFaced           0
winner_rank         0
loser_rank          0
dtype: int64

Seguimos con las variables 'winner_ht' y 'loser_ht'. En este caso imputamos la media del resto de valores.

In [59]:
df['winner_ht'] = df['winner_ht'].transform(lambda x: x.fillna(x.mean()))
df['loser_ht'] = df['loser_ht'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()

tourney_id       0
tourney_name     0
surface          0
draw_size        0
tourney_level    0
tourney_date     0
match_num        0
winner_id        0
winner_name      0
winner_hand      1
winner_ht        0
winner_ioc       0
winner_age       0
loser_id         0
loser_name       0
loser_hand       5
loser_ht         0
loser_ioc        0
loser_age        9
score            0
best_of          0
round            0
minutes          0
w_ace            0
w_df             0
w_svpt           0
w_1stIn          0
w_1stWon         0
w_2ndWon         0
w_SvGms          0
w_bpSaved        0
w_bpFaced        0
l_ace            0
l_df             0
l_svpt           0
l_1stIn          0
l_1stWon         0
l_2ndWon         0
l_SvGms          0
l_bpSaved        0
l_bpFaced        0
winner_rank      0
loser_rank       0
dtype: int64

Por último, nos fijamos en los atributos 'winner_hand', 'loser_hand' y 'loser_age'. Para los primeros sustituimos los NaN por 'U' ('unknown') y para el tercero imputamos la media de edad del resto de jugadores, ya que hay poca desviación en la distribución de los datos (los jugadores tienden a moverse entre un rango concreto de edad). 

In [60]:
df['winner_hand'] = df['winner_hand'].fillna('U')
df['loser_hand'] = df['loser_hand'].fillna('U')
df['loser_age'] =  df['loser_age'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

tourney_id       0
tourney_name     0
surface          0
draw_size        0
tourney_level    0
tourney_date     0
match_num        0
winner_id        0
winner_name      0
winner_hand      0
winner_ht        0
winner_ioc       0
winner_age       0
loser_id         0
loser_name       0
loser_hand       0
loser_ht         0
loser_ioc        0
loser_age        0
score            0
best_of          0
round            0
minutes          0
w_ace            0
w_df             0
w_svpt           0
w_1stIn          0
w_1stWon         0
w_2ndWon         0
w_SvGms          0
w_bpSaved        0
w_bpFaced        0
l_ace            0
l_df             0
l_svpt           0
l_1stIn          0
l_1stWon         0
l_2ndWon         0
l_SvGms          0
l_bpSaved        0
l_bpFaced        0
winner_rank      0
loser_rank       0
dtype: int64

In [61]:
df.dtypes

tourney_id        object
tourney_name      object
surface           object
draw_size          int64
tourney_level     object
tourney_date       int64
match_num          int64
winner_id          int64
winner_name       object
winner_hand       object
winner_ht        float64
winner_ioc        object
winner_age       float64
loser_id           int64
loser_name        object
loser_hand        object
loser_ht         float64
loser_ioc         object
loser_age        float64
score             object
best_of            int64
round             object
minutes          float64
w_ace            float64
w_df             float64
w_svpt           float64
w_1stIn          float64
w_1stWon         float64
w_2ndWon         float64
w_SvGms          float64
w_bpSaved        float64
w_bpFaced        float64
l_ace            float64
l_df             float64
l_svpt           float64
l_1stIn          float64
l_1stWon         float64
l_2ndWon         float64
l_SvGms          float64
l_bpSaved        float64


Una vez tratados todos los NaNs, nos fijamos en el tipo de variable. Es necesario hacer los siguientes cambios:
- 'tourney_date': transformarlo al formato fecha y extraer el año.  
- 'match_num', 'winner_id', 'loser_id': convertirlos en objectos, ya que son identificadores y no sirven para hacer operaciones aritméticas. Si se guardan como objetos ocupan menos memoria.
- Convertir las variables que se almacenan como decimales cuando realmente son números enteros.

In [62]:
df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df['year'] = df['tourney_date'].dt.year
df.dtypes

tourney_id               object
tourney_name             object
surface                  object
draw_size                 int64
tourney_level            object
tourney_date     datetime64[ns]
match_num                 int64
winner_id                 int64
winner_name              object
winner_hand              object
winner_ht               float64
winner_ioc               object
winner_age              float64
loser_id                  int64
loser_name               object
loser_hand               object
loser_ht                float64
loser_ioc                object
loser_age               float64
score                    object
best_of                   int64
round                    object
minutes                 float64
w_ace                   float64
w_df                    float64
w_svpt                  float64
w_1stIn                 float64
w_1stWon                float64
w_2ndWon                float64
w_SvGms                 float64
w_bpSaved               float64
w_bpFace

Guardamos el dataset limpio en un archivo csv.

In [64]:
df.to_csv('clean_data.csv')